In [1]:
import requests
import os
import pandas as pd

In [2]:
modes = ['dlr', 'national-rail', 'overground', 'tflrail', 'tube']

In [3]:
class TflStopPoint:
    def __init__(self,mode):
        self.mode = mode
        
    @property
    def endpoint(self):
        return f"https://api.tfl.gov.uk/StopPoint/Mode/{self.mode}"
    
    @property
    def stopPoint_json(self):
        return requests.get(self.endpoint).json()
    
    @property
    def stopPoint_data(self):
        # dict items to retrieve
        desired_keys = [ 'indicator','naptanId', 'stationNaptan', 'lines', 'lineGroup', 'placeType'
                       , 'stopType', 'commonName', 'modes', 'lat', 'lon']

        data=[]
        for i, value in enumerate(self.stopPoint_json['stopPoints']):
            
            stopPoint = {}

            for key in desired_keys:
                if key in value:
                    stopPoint[key] = value[key]
                else:
                    stopPoint[key] = None

            for dict_obj in value['additionalProperties']:
                for key in dict_obj:
                    if key == 'key':
                        stopPoint[dict_obj[key]] = {'value' : dict_obj['value'], 'category' : dict_obj['category']}

            data.append(stopPoint)
        return data
    

In [4]:
x = TflStopPoint('tube')

In [5]:
x.endpoint

'https://api.tfl.gov.uk/StopPoint/Mode/tube'

In [6]:
pd.DataFrame(x.stopPoint_data).head(12)

,ASDA Click and Collect,AccessViaLift,Address,AddtionalInformation,Amazon Lockers,BlueBadgeCarParkSpaces,Boarding Ramp,Boarding Ramps,Bridge,Car park,...,indicator,lat,lineGroup,lines,lon,modes,naptanId,placeType,stationNaptan,stopType
0,NaN,"{'value': 'No', 'category': 'Accessibility'}","{'value': 'Amersham Station,Stn Approach,Amers...",{'value': 'You can only enter and exit the sou...,NaN,"{'value': '0', 'category': 'Accessibility'}","{'value': 'yes', 'category': 'Facility'}",NaN,"{'value': 'yes', 'category': 'Facility'}","{'value': 'yes', 'category': 'Facility'}",...,main entrance,51.674203,[],[],-0.607365,[tube],0400ZZLUAMS0,StopPoint,940GZZLUAMS,NaptanMetroEntrance
1,NaN,"{'value': 'No', 'category': 'Accessibility'}","{'value': 'Chalfont & Latimer Station,London U...",NaN,NaN,"{'value': '5', 'category': 'Accessibility'}",NaN,NaN,"{'value': 'no', 'category': 'Facility'}","{'value': 'yes', 'category': 'Facility'}",...,South Entrance,51.667912,[],[],-0.560619,[tube],0400ZZLUCAL0,StopPoint,940GZZLUCAL,NaptanMetroEntrance
2,NaN,"{'value': 'No', 'category': 'Accessibility'}","{'value': 'Chalfont & Latimer Station,London U...",NaN,NaN,"{'value': '5', 'category': 'Accessibility'}",NaN,NaN,"{'value': 'no', 'category': 'Facility'}","{'value': 'yes', 'category': 'Facility'}",...,North Entrance,51.668119,[],[],-0.560627,[tube],0400ZZLUCAL1,StopPoint,940GZZLUCAL,NaptanMetroEntrance
3,NaN,"{'value': 'No', 'category': 'Accessibility'}","{'value': 'Chesham Station,London Underground ...",NaN,NaN,"{'value': '4', 'category': 'Accessibility'}",NaN,NaN,"{'value': 'no', 'category': 'Facility'}","{'value': 'yes', 'category': 'Facility'}",...,main entrance,51.705224,[],[],-0.611116,[tube],0400ZZLUCSM0,StopPoint,940GZZLUCSM,NaptanMetroEntrance
4,NaN,NaN,"{'value': 'Buckhurst Hill Station,London Under...",NaN,NaN,NaN,NaN,NaN,"{'value': 'yes', 'category': 'Facility'}","{'value': 'yes', 'category': 'Facility'}",...,entrance,51.626627,[],[],0.046498,[tube],1500ZZLUBKH0,StopPoint,940GZZLUBKH,NaptanMetroEntrance
5,NaN,NaN,"{'value': 'Buckhurst Hill Station,London Under...",NaN,NaN,NaN,NaN,NaN,"{'value': 'yes', 'category': 'Facility'}","{'value': 'yes', 'category': 'Facility'}",...,east entrance,51.625233,[],[],0.047042,[tube],1500ZZLUBKH1,StopPoint,940GZZLUBKH,NaptanMetroEntrance
6,NaN,NaN,"{'value': 'Buckhurst Hill Station,London Under...",NaN,NaN,NaN,NaN,NaN,"{'value': 'yes', 'category': 'Facility'}","{'value': 'yes', 'category': 'Facility'}",...,west entrance,51.625320,[],[],0.046179,[tube],1500ZZLUBKH2,StopPoint,940GZZLUBKH,NaptanMetroEntrance
7,NaN,NaN,"{'value': 'Chigwell Station,London Underground...",NaN,NaN,NaN,NaN,NaN,"{'value': 'no', 'category': 'Facility'}","{'value': 'no', 'category': 'Facility'}",...,south entrance,51.617858,[],[],0.074258,[tube],1500ZZLUCWL0,StopPoint,940GZZLUCWL,NaptanMetroEntrance
8,NaN,"{'value': 'No', 'category': 'Accessibility'}","{'value': 'Debden Station,London Underground L...",{'value': 'You can only enter and exit the eas...,NaN,"{'value': '3', 'category': 'Accessibility'}",NaN,NaN,"{'value': 'yes', 'category': 'Facility'}","{'value': 'yes', 'category': 'Facility'}",...,north entrance,51.645557,[],[],0.083804,[tube],1500ZZLUDBN0,StopPoint,940GZZLUDBN,NaptanMetroEntrance
9,NaN,"{'value': 'No', 'category': 'Accessibility'}","{'value': 'Debden Station,London Underground L...",{'value': 'You can only enter and exit the eas...,NaN,"{'value': '3', 'category': 'Accessibility'}",NaN,NaN,"{'value': 'yes', 'category': 'Facility'}","{'value': 'yes', 'category': 'Facility'}",...,south entrance,51.645156,[],[],0.084104,[tube],1500ZZLUDBN1,StopPoint,940GZZLUDBN,NaptanMetroEntrance


In [7]:
pd.DataFrame(x.stopPoint_data).columns

Index(['ASDA Click and Collect', 'AccessViaLift', 'Address',
       'AddtionalInformation', 'Amazon Lockers', 'BlueBadgeCarParkSpaces',
       'Boarding Ramp', 'Boarding Ramps', 'Bridge', 'Car park',
       'Cash Machines', 'Escalators', 'Euro Cash Machines', 'Gates',
       'Help Points', 'Lifts', 'LimitedCapacityLift', 'Location', 'MonFriFrom',
       'MonFriTo', 'Night', 'Other Facilities', 'Payphones', 'PhoneNo',
       'Photo Booths', 'SatFrom', 'SatTo', 'SourceSystemPlaceId',
       'SpecificEntranceInstructions', 'SpecificEntranceRequired', 'SunFrom',
       'SunTo', 'TaxiRankOutsideStation', 'Ticket Halls', 'Toilet',
       'ToiletNote', 'Toilets', 'Waiting Room', 'WiFi', 'Zone', 'commonName',
       'indicator', 'lat', 'lineGroup', 'lines', 'lon', 'modes', 'naptanId',
       'placeType', 'stationNaptan', 'stopType'],
      dtype='object')

In [9]:
pd.DataFrame(x.stopPoint_data).dtypes

ASDA Click and Collect           object
AccessViaLift                    object
Address                          object
AddtionalInformation             object
Amazon Lockers                   object
BlueBadgeCarParkSpaces           object
Boarding Ramp                    object
Boarding Ramps                   object
Bridge                           object
Car park                         object
Cash Machines                    object
Escalators                       object
Euro Cash Machines               object
Gates                            object
Help Points                      object
Lifts                            object
LimitedCapacityLift              object
Location                         object
MonFriFrom                       object
MonFriTo                         object
Night                            object
Other Facilities                 object
Payphones                        object
PhoneNo                          object
Photo Booths                     object


In [8]:
x.stopPoint_json['stopPoints'][0]

{'$type': 'Tfl.Api.Presentation.Entities.StopPoint, Tfl.Api.Presentation.Entities',
 'naptanId': '0400ZZLUAMS0',
 'indicator': 'main entrance',
 'stopLetter': 'entrance',
 'modes': ['tube'],
 'icsCode': '1000006',
 'stopType': 'NaptanMetroEntrance',
 'stationNaptan': '940GZZLUAMS',
 'hubNaptanCode': 'HUBAMR',
 'lines': [],
 'lineGroup': [],
 'lineModeGroups': [],
 'status': True,
 'id': '0400ZZLUAMS0',
 'commonName': 'Amersham Underground Station',
 'placeType': 'StopPoint',
 'additionalProperties': [{'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Accessibility',
   'key': 'TaxiRankOutsideStation',
   'sourceSystemKey': 'LRAD',
   'value': 'No'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Accessibility',
   'key': 'Toilet',
   'sourceSystemKey': 'LRAD',
   'value': 'Yes'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentati